Задание:
1.	Загрузить файл длиной не менее 2000 символов. 
2.	Составить программу, которая считает число уникальных слов в тексте (без критерия схожести)
3.	Составить программу, которая считает число гласных и согласных букв. 
4.	Составить программу, которая считает число предложений, их длину и число (количество) раз использования каждого слова в тексте (с критерием схожести, критерий схожести слов выбрать самостоятельно, например, spacy (en_core_web_sm) или расстояние Левенштейна). 
5.	Вывести 10 наиболее часто встречаемых слов. 

***
В данном блокноте выполняем пункт 4 и 5 задания.

***
Анализ по данным пунктам задания:
a) Для подсчета числа предложений можно находить знаки конца предложения (точку, вопросительный и восклицательный знак). Если между предыдущим таким знаком (или началом текста) и текущим знаком находится хотя бы один алфавитно-цифровой знак, то это и будет предложение.
Таким образом, можно разбить текст на список предложений. Длина списка будет числом предложений в тексте.
b) Для отыскания длины каждого предложения можно предварительно очистить каждое предложение в списке от знаков препинания, заменив их пробелами, и расщепить на список слов с помощью соответствующей процедуры Python. Длина этого списка будет длиной предложения.
c) Для нахождения числа раз использования каждого слова в тексте с использованием критерия схожести используем библиотеку spacy. 
Вначале мы уберем из текста все знаки препинания и разделим текст на отдельные слова. Получим список всех слов текста. Используя процедуру, отработанную на одном из предыдущих этапов (при выполнении второго пункта данного задания), уберем незначащие слова из списка.
Список будет включать все значащие слова,  причем столько раз, сколько имеется вхождений этого слова в исходный текст.
Простым преобразованием можно получить множество всех слов текста, где по определению каждое слово входит только одина раз.
Далее для каждого слова из множества, используя функции библиотеки spacy, можно найти в этом множестве список похожих на него слов. 
Эта процедура дает набор списков похожих слов, из которых состоит текст. Для каждого слова каждого списка можно найти количество его вхождений в очищенный текст. Взяв сумму числа вхождений для слов каждого списка, можно получить искомое число раз использования каждого слова в тексте.
Чтобы не было путаницы с регистрами текста, предварительно можно привести весь текст к нижнему регистру.

Тогда план работы по пункту 2 следующий.

1. Загрузить текст из файла.
2. Перевести текст в нижний регистр.
3. Проходить строку, включающую исследуемый текст, знак за знаком, находить знак конца предложения, то есть точку, восклицательный или положительный знак, и его положение в тексте. Для точки проверять, чтобы справа и слева не было цифр, иначе это будет десятичная точка числа, а не знак конца предложения.
4. Взяв слайс между предыдущим и текущим знаком конца предложения, проверить, есть ли в этом слайсе алфавитно-цифровые знаки. В случае наличия считаем полученный слайс очередным предложением текста. 
5. Далее из этого предложения создать новую строку, включающую только алфавитно-цифровые знаки, остальные знаки заменить пробелами. Затем расщепить полученную строку процедурой split и найти длину полученного списка. Это будет длина предложения, измеренная в словах. Пройдя таким образом все предложения текста, вывести  результат - список длин всех предложений - на экран.
6. Убрать в тексте, полученном в пункте 2, знаки препинания и шире - все неалфавитные знаки, заменив их пробелами.
7. Разбить текст на слова процедурой split, считая пробелы разделителями.
8. Из полученного списка слов убрать все стоп-слова. Получить тем самым список значащих слов текста.
9. Получить множество уникальных (пока без критерия схожести) слов текста, преобразовав список в множество.
10. Загрузить модель spacy en_core_web_sm. Для каждого слова множества значащих с помощью данной модели найти схожие с ним слова и число их вхождений в очищенный текст. Также занести в словарь номер каждой группы схожих слов и число упоминаний слов данной группы.
11. Отсортировать полученный словарь по значению ключа (дающего частотность соответствующей группы схожих слов)
12. Отобрать 10 наиболее часто встречающихся групп слов в полученном словаре.
14. Если в полученном словаре есть еще группы с той же частотностью, что и у десятой группы, также включить их в выводимый результат.

1. Загружаем текстовый файл:

In [93]:
import spacy

file_name = 'GospelJohn.txt' # file with text
# file_name = 'example.txt' # file with text
try:
    f = open(file_name,"r") # open file for reading
    text = f.read()         # reading file 
    f.close()               # closing file
    print('Read successfully')
except:
    print('Error reading file!')

Read successfully


2. Переводим текст в нижний регистр.

In [94]:
# to lower case:
text = text.lower()

3. Проходим строку, включающую исследуемый текст, знак за знаком, находим знак конца предложения, то есть точку, восклицательный или положительный знак, и его положение в тексте. Для точки проверяем, чтобы справа и слева не было цифр, иначе это будет десятичная точка числа, а не знак конца предложения.
4. Взяв слайс между предыдущим и текущим знаком конца предложения, проверяем, есть ли в этом слайсе алфавитно-цифровые знаки. В случае наличия считаем полученный слайс очередным предложением текста. 
5. Далее из этого предложения создаем новую строку, включающую только алфавитно-цифровые знаки, остальные знаки заменяем пробелами. Затем расщепляем полученную строку процедурой split и находим длину полученного списка. Это будет длина предложения, измеренная в словах. Пройдя таким образом все предложения текста, выводим  результат - список длин всех предложений - на экран.

In [95]:
# "left" - variable for left-hand end of a sentence
right = -1 # variable for right-hand end of a sentence
i = -1 # counter
sentences_length_list = list() # list for sentences lengths
for c in text:
    i += 1
    if c in ".?!":
        if c == '.' and text[i-1].isnumeric() and text[i+1].isnumeric():
            continue
        # new boundaries of a string:
        left = right + 1
        right = i
        astring = text[left:right + 1] # extract the string
        # print(f'String:\n{astring}')
        # if the string contains alphanumeric symbols, we suppose for it to be a sentence
        if len([ch for ch in astring if ch.isalnum()]) > 0:
            sentence_alphanumeric = "".join([ch if ch.isalnum else " " for ch in astring])
            sentence_length = len(sentence_alphanumeric.split()) # length of sentence in words
            sentences_length_list.append(sentence_length)
print(f'Lengths of sentences in the text (in words):\n {sentences_length_list}')
print(f'Number of sentences in the text:\n {len(sentences_length_list)}')
print(f'Average length of sentences in the text (in words):\n {round(sum(sentences_length_list) / len(sentences_length_list),1)}')

Lengths of sentences in the text (in words):
 [17, 7, 15, 13, 13, 11, 17, 15, 13, 19, 15, 42, 10, 23, 4, 27, 15, 14, 28, 21, 14, 8, 3, 6, 5, 4, 7, 12, 6, 27, 27, 16, 19, 17]
Number of sentences in the text:
 34
Average length of sentences in the text (in words):
 15.0


6. Убираем в тексте, полученном в пункте 2, знаки препинания и шире - все неалфавитные знаки, заменив их пробелами.

In [96]:
# replace non-alphanumeric signs by whitespaces:
text_no_signs = "".join([c if c.isalnum() else " " for c in text])

7. Разбиваем текст на слова процедурой split, считая пробелы разделителями.

In [97]:
word_list = text_no_signs.split() # split text into words and put to the list
print(word_list)

['in', 'the', 'beginning', 'was', 'the', 'word', 'and', 'the', 'word', 'was', 'with', 'god', 'and', 'the', 'word', 'was', 'god', 'he', 'was', 'with', 'god', 'in', 'the', 'beginning', 'through', 'him', 'all', 'things', 'were', 'made', 'without', 'him', 'nothing', 'was', 'made', 'that', 'has', 'been', 'made', 'in', 'him', 'was', 'life', 'and', 'that', 'life', 'was', 'the', 'light', 'of', 'all', 'mankind', 'the', 'light', 'shines', 'in', 'the', 'darkness', 'and', 'the', 'darkness', 'has', 'not', 'overcome', 'it', 'there', 'was', 'a', 'man', 'sent', 'from', 'god', 'whose', 'name', 'was', 'john', 'he', 'came', 'as', 'a', 'witness', 'to', 'testify', 'concerning', 'that', 'light', 'so', 'that', 'through', 'him', 'all', 'might', 'believe', 'he', 'himself', 'was', 'not', 'the', 'light', 'he', 'came', 'only', 'as', 'a', 'witness', 'to', 'the', 'light', 'the', 'true', 'light', 'that', 'gives', 'light', 'to', 'everyone', 'was', 'coming', 'into', 'the', 'world', 'he', 'was', 'in', 'the', 'world', '

8. Из полученного списка слов убираем все стоп-слова. Получаем тем самым список значащих слов текста.

Для этого надо вначале создать список стоп-слов
(источник: https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py)

In [98]:
STOP_WORDS = set(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at

back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by

call can cannot ca could

did do does doing done down due during

each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except

few fifteen fifty first five for former formerly forty four from front full
further

get give go

had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred

i if in indeed into is it its itself

keep

last latter latterly least less

just

made make many may me meanwhile might mine more moreover most mostly move much
must my myself

name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere

of off often on once one only onto or other others otherwise our ours ourselves
out over own

part per perhaps please put

quite

rather re really regarding

s same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such 

take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two

under until up unless upon us used using

various very very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would

yet you your yours yourself yourselves
""".split()
)

contractions = ["n't", "nt", "'d", "d", "'ll", "ll", "'m", "m", "'re", "re", "'s", "s", "'ve", "ve"]
STOP_WORDS.update(contractions)

for apostrophe in ["‘", "’"]:
    for stopword in contractions:
        STOP_WORDS.add(stopword.replace("'", apostrophe))

obsoletisms = ["art", "hath", "shalt", "thou", "thee", "thyself", "ye", "ya", "unto"] # obsolete word forms
STOP_WORDS.update(obsoletisms)

print(STOP_WORDS)

{'in', "'m", 'afterwards', 'they', 'any', 'really', 'first', 'wherein', 'hereupon', 'does', 'had', 'eleven', 'cannot', 'itself', 'must', 'last', 'although', 'did', 'whoever', '’re', 'yourself', 'she', 'again', 'of', 'sixty', '‘ve', 'few', 'made', 'always', 'will', 'upon', 'with', 'thru', 'could', 'ye', 'no', 'without', 'though', 'only', 'him', 'more', 'call', 'toward', 'there', 'll', 'his', 'whereafter', 'seemed', 'yet', 'another', '’d', 'me', 'you', 'back', 's', 'shalt', 'sometime', 'hundred', 'perhaps', 'a', 'becomes', 'below', 'hereby', 'already', 'none', 'rather', 'make', 'whose', '‘s', 'either', 'so', 'us', 'therein', 'others', 'd', 'mine', 'via', 'empty', 'say', 'hath', 'three', 'do', 'may', 'been', '’ll', 'but', 'art', 'i', '‘d', 'third', 'moreover', 'well', 'when', 'go', 'ca', 'whatever', 'against', 'five', 'per', 'doing', 'then', 'because', 'everything', 'what', 'latterly', 'thereby', 'never', 'down', 'much', 'twelve', 'whenever', 'besides', 'now', 'myself', 'whence', 'fifty',

Продолжение п.8. 
Убираем из списка слова, если они входят в список стоп-слов:

In [99]:
word_list_new = list()
for aword in word_list:
    if aword not in STOP_WORDS:
        word_list_new.append(aword)

word_list = word_list_new
print(word_list)

['beginning', 'word', 'word', 'god', 'word', 'god', 'god', 'beginning', 'things', 'life', 'life', 'light', 'mankind', 'light', 'shines', 'darkness', 'darkness', 'overcome', 'man', 'sent', 'god', 'john', 'came', 'witness', 'testify', 'concerning', 'light', 'believe', 'light', 'came', 'witness', 'light', 'true', 'light', 'gives', 'light', 'coming', 'world', 'world', 'world', 'world', 'recognize', '11he', 'came', 'receive', 'receive', 'believed', 'gave', 'right', 'children', 'god', 'children', 'born', 'natural', 'descent', 'human', 'decision', 'husband', 'born', 'god', 'word', 'flesh', 'dwelling', 'seen', 'glory', 'glory', 'son', 'came', 'father', 'grace', 'truth', 'john', 'testified', 'concerning', 'cried', 'saying', 'spoke', 'said', 'comes', 'surpassed', 'fullness', 'received', 'grace', 'place', 'grace', 'given', 'law', 'given', 'moses', 'grace', 'truth', 'came', 'jesus', 'christ', 'seen', 'god', 'son', 'god', 'closest', 'relationship', 'father', 'known', 'john', 'testimony', 'jewish', 

9. Получаем множество уникальных (пока без критерия схожести) слов текста, преобразовав список в множество.

In [100]:
word_set = set(word_list)
print(word_set)

{'testify', 'sandals', 'closest', 'right', 'said', 'sent', 'calling', 'relationship', 'happened', 'water', 'leaders', 'jordan', 'father', 'fullness', 'true', 'gave', 'john', '11he', 'cried', 'testified', 'decision', 'flesh', 'finally', 'know', 'saying', 'dwelling', 'god', 'grace', 'untie', 'coming', 'bethany', 'human', 'levites', 'children', 'received', 'comes', 'known', 'replied', 'voice', 'fail', 'straps', 'witness', 'freely', 'way', 'descent', 'isaiah', 'testimony', 'worthy', 'baptizing', 'world', 'words', 'life', 'truth', 'asked', 'stands', 'jerusalem', 'born', 'things', 'recognize', 'moses', 'man', 'concerning', 'beginning', 'surpassed', 'jewish', 'straight', 'son', 'mankind', 'jesus', 'messiah', 'answer', 'seen', 'given', 'prophet', 'husband', 'confess', 'darkness', 'believed', 'word', 'gives', 'ask', 'spoke', 'light', 'pharisees', 'baptize', 'elijah', 'came', 'confessed', 'priests', 'receive', 'natural', 'law', 'answered', 'wilderness', 'questioned', 'overcome', 'glory', 'believ

10. Загружаем модель spacy en_core_web_md. 
Для каждого слова множества значащих с помощью данной модели находим схожие с ним слова и число их вхождений в очищенный текст. 
Эти похожие слова заносим в список проверенных слов, чтобы на следующей итерации не считать их снова.
Повторяем для каждого следующего слова из множества значащих слов.
Попутно заносим в словарь номер каждой группы схожих слов и число упоминаний слов данной группы.
Выводим на экран группы схожих слов и число упоминаний слов этой группы в тексте.

In [101]:
# Loading the large-sized spacy English model (to embrace all the terms in the text)
nlp = spacy.load('en_core_web_md')
# Creating a SpaCy document. Since the argument has to be a string, 
# we have to make a string from the prepared word set first:
doc_spacy = nlp(" ".join([aword for aword in word_list]))  

word_groups = list()
current_word_set = set()
frequency_dict = dict()
words_checked = set()
i = 0 # word group counter
for aword in word_set:
    if aword not in words_checked:
        print(f'The word being analyzed: {aword}')
        aword_nlp = nlp(aword) # 
        j = 0 # words in a group counter
        for w in doc_spacy:            
            if w.has_vector and aword_nlp.similarity(w) > 0.9:  # checking for similarity; 0.x is similarity threshold
                    print(f'Checking {aword_nlp.text} and {w.text}')
                    current_word_set.add(w.text) 
                    words_checked.add(w.text)           
                    j += 1
        word_groups.append(current_word_set) # appending the found set of similar words to global list
        frequency_dict[i] = j
        print('Current word set:')
        print(', '.join([w for w in current_word_set]))        
        print(f'Number of these words in the text:\n {j}\n')
        current_word_set = set() # init the current set again
        i += 1 # increase counter
    
# print(f"Word set (to be empty): \n {word_set}")
# print(f"Word groups: \n {word_groups}")
# print(f'{frequency_dict}')
print(f"Word group quantity: \n {len(word_groups)}")

The word being analyzed: testify
Checking testify and testify
Current word set:
testify
Number of these words in the text:
 1

The word being analyzed: sandals
Checking sandals and sandals
Current word set:
sandals
Number of these words in the text:
 1

The word being analyzed: closest
Checking closest and closest
Current word set:
closest
Number of these words in the text:
 1

The word being analyzed: right
Checking right and things
Checking right and right
Checking right and fail
Checking right and straight
Checking right and way
Checking right and know
Current word set:
straight, way, know, things, fail, right
Number of these words in the text:
 6

The word being analyzed: said
Checking said and came
Checking said and came
Checking said and came
Checking said and gave
Checking said and came
Checking said and cried
Checking said and spoke
Checking said and said
Checking said and came
Checking said and asked
Checking said and said
Checking said and said
Checking said and replied
Check

C:\Users\User\AppData\Local\Temp\ipykernel_19352\3345262030.py:18: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  if w.has_vector and aword_nlp.similarity(w) > 0.9:  # checking for similarity; 0.x is similarity threshold


Checking flesh and flesh
Current word set:
flesh
Number of these words in the text:
 1

The word being analyzed: finally
Checking finally and gives
Checking finally and comes
Checking finally and finally
Checking finally and comes
Current word set:
finally, comes, gives
Number of these words in the text:
 4

The word being analyzed: saying
Checking saying and saying
Current word set:
saying
Number of these words in the text:
 1

The word being analyzed: dwelling
Checking dwelling and dwelling
Current word set:
dwelling
Number of these words in the text:
 1

The word being analyzed: god
Checking god and god
Checking god and god
Checking god and god
Checking god and god
Checking god and god
Checking god and god
Checking god and god
Checking god and god
Current word set:
god
Number of these words in the text:
 8

The word being analyzed: grace
Checking grace and glory
Checking grace and glory
Checking grace and grace
Checking grace and grace
Checking grace and grace
Checking grace and gra

11. Сортируем полученный словарь по значению ключа (дающего частотность соответствующей группы схожих слов)
12. Отбираем 10 наиболее часто встречающихся групп слов в полученном словаре.
13. Если в полученном словаре есть еще группы с той же частотностью, что и у десятой группы, также включаем их в выводимый результат. 

In [102]:
frequency_dict_sorted = sorted(frequency_dict.items(), key = lambda item: item[1])
print('The most frequent word groups:')

# extracting the most frequent words
if len(word_groups) > 10:
    max_num = 10
else:
    max_num = len(word_groups)

for i in range(max_num):
    group_num = frequency_dict_sorted[-i-1][0]
    word_group = word_groups[group_num]
    frequency = frequency_dict_sorted[-i-1][1]
    print(f'{i+1}. The word group\n"{word_group}"\n - has the frequency {frequency}')

# extracting also the 11th, 12th etc, if the frequency is the same as for the 10th one:
j = 1
while (i+j < len(word_groups)-2) and (frequency_dict_sorted[-i-j-1][1] == frequency_dict_sorted[-i-1][1]):
    group_num = frequency_dict_sorted[-i - j - 1][0]
    word_group = word_groups[group_num]
    frequency = frequency_dict_sorted[-i - j -1][1]
    print(f'{i + j + 1}. The word group\n"{word_group}"\n - has the frequency {frequency}')
    j += 1

The most frequent word groups:
1. The word group
"{'said', 'gave', 'spoke', 'replied', 'asked', 'came', 'cried'}"
 - has the frequency 14
2. The word group
"{'god'}"
 - has the frequency 8
3. The word group
"{'light'}"
 - has the frequency 7
4. The word group
"{'world', 'life'}"
 - has the frequency 6
5. The word group
"{'glory', 'grace'}"
 - has the frequency 6
6. The word group
"{'john'}"
 - has the frequency 6
7. The word group
"{'straight', 'way', 'know', 'things', 'fail', 'right'}"
 - has the frequency 6
8. The word group
"{'recognize', 'sent'}"
 - has the frequency 5
9. The word group
"{'word'}"
 - has the frequency 4
10. The word group
"{'christ', 'pharisees', 'jesus', 'priests'}"
 - has the frequency 4
11. The word group
"{'moses', 'isaiah', 'elijah'}"
 - has the frequency 4
12. The word group
"{'finally', 'comes', 'gives'}"
 - has the frequency 4
13. The word group
"{'witness', 'testimony', 'testified'}"
 - has the frequency 4
14. The word group
"{'seen', 'coming', 'happened'}

Итак, нами был проделан анализ англоязычного текста в отношении количества слов в предложениях, а также количества уникальных слов в тексте с объединением слов по критерию схожести.

Выводы:
1. В анализируемом англоязычном тексте тексте на ~500 слов имеется 34 предложений, в среднем по 15 слов в каждом.
2. В данном тексте после удаления стоп-слов и объединения оставшихся по принципу схожести оказалось 69 групп слов. Таким образом, отдельных групп оказалось примерно на порядок меньше,  чем общее число всех слов (499).
3. Десять отобранных уникальных слов с максимальной частотностью дают хорошее представление о тематике текста.
4. Объединение слов в группы по критерию схожести, применяемому в библиотеке spacy, дает иногда вполне ожидаемые результаты (объединение в группу слов glory и grace), так и несколько неожиданные объединения слов в группу (напр. 'seen', 'coming', 'happened')
5. Эксперименты с различными текстами, в том числе устаревшими, показали, что анализ работает без сбоев, только если загруженная языковая модель spacy включает все слова текста. Таким образом, следует учитывать тезаурус модели, который должен быть шире, чем совокупность слов анализируемого текста.